In [1]:
print ('IOT device log file parsing...')


IOT device log file parsing...


In [2]:
import fileinput
import json
import pandas as pd

#location of input file which contains many log message. we are parsing subset of JSON messages
LOG_PATH = 'D:\\a_2018\\2019\\sd_global_b\\ven_out.txt'

#outfile
outFile = 'D:\\a_2018\\2019\\sd_global_b\\ven_lng.txt'

#list of attributes
mylist = ['vin','lat','lng']

##for processing nested JSON for some keys
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k in key:
                    #print (v) 
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

# process the incoming log file..
###log file is supposed to contains location of devie after CreateTime token
def process(line):
   valid = 'CreateTime'
   length = len(valid)
   pos = line.find(valid)
   if ( pos > 0):
       msg = line[pos+length:]  ### pick entier string after CreateTime
       data = json.loads(msg)
       return data


df = pd.DataFrame(columns=['VIN','LAT','LNG'])

for line in fileinput.FileInput(LOG_PATH):
    data = process(line)
    if (data):
        names = extract_values( data , mylist)
        if ( len(names) == 3):
            vin=names[0]
            lat=names[1]
            lng=names[2]
            aRow = pd.Series(data={'VIN':vin,'LAT':lat,'LNG':lng})
            df = df.append(aRow,ignore_index=True)
fileinput.close()   


In [3]:
df.shape

(47, 3)

In [7]:
import gmplot
#import gmaps

## convert str to flots. 

df['LAT'] = df['LAT'].astype(float)
df['LNG'] = df['LNG'].astype(float)
latitudes = df["LAT"]
longitudes = df["LNG"]

#locations = df["LAT","LNG"]

#fig = gmaps.figure(map_type='HYBRID')
#heatmap_layer = gmaps.heatmap_layer(locations)
#fig.add_layer(heatmap_layer)
#fig

gmap = gmplot.GoogleMapPlotter(46.0436,  -73.1129, 10) #some base point
gmap.heatmap(latitudes, longitudes)
gmap.draw("D:\\a_2018\\map15.html")

In [ ]:
### TML file was generated. Atatch it